In [1]:
import pandas as pd
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    #model_name = "ai-forever/sbert_large_mt_nlu_ru",
    model_name = "sergeyzh/rubert-mini-uncased",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True} # i wanted to explicit set this to False since we are using weaviate, but DONT FORGET TO SET IT TO TRUE if we stop using weaviate. SET IT TO TRUE if vector db doesnt normalize automatically
)



/home/AlexxFlexing/RAG/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader #not sure about those 2
from langchain_community.document_loaders.csv_loader import CSVLoader #i mean this as second
from langchain_weaviate.vectorstores import WeaviateVectorStore
import weaviate
import weaviate.classes as wvc
from weaviate.classes.config import Configure, Property, DataType

#series_result = '../data/processed/series_results.csv'
#df = pd.read_csv(series_result)

client = weaviate.connect_to_local(
    host="127.0.0.1",  # Use a string to specify the host
    port=8080,
    grpc_port=50051,
)

if client.collections.exists("MatchData"): #redo this in prod obviously
    client.collections.delete("MatchData")

if client.collections.exists("MapStats"):
    client.collections.delete("MapStats")

client.collections.create(
    name="MatchData",
    properties=[
        Property(name="team_id", data_type=DataType.TEXT),
        Property(name="team_name", data_type=DataType.TEXT),
        Property(name="date", data_type=DataType.TEXT),
        Property(name="opponent_id", data_type=DataType.TEXT),
        Property(name="opponent_name", data_type=DataType.TEXT),
        Property(name="map_name", data_type=DataType.TEXT),
        Property(name="series_result", data_type=DataType.TEXT),
        Property(name="description", data_type=DataType.TEXT),
    ],
    description="Информация о матче между командами",
    vectorizer_config=Configure.Vectorizer.none()  # for manual embedding
)

# Create MapStats class
client.collections.create(
    name="MapStats",
    properties=[
        Property(name="team_id", data_type=DataType.TEXT),
        Property(name="map_name", data_type=DataType.TEXT),
        Property(name="winrate", data_type=DataType.NUMBER),
        Property(name="pickrate", data_type=DataType.NUMBER),
        Property(name="banrate", data_type=DataType.NUMBER),
    ],
    description="Статистика по картам для команд",
    vectorizer_config=Configure.Vectorizer.none()
)


USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/AlexxFlexing/RAG/venv/lib/python3.12/site-packages/weaviate/warnings.py:340: UserWarning: Con006: You're using the sync client in an async context. This usage is discouraged to avoid blocking your async event loop with sync I/O calls.
            We encourage you to update your code to use the async client instead when running inside async def functions!
  warnings.warn(


In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("sergeyzh/rubert-mini-uncased")

series_result = '../data/processed/series_results.csv'
maps_stats = '../data/processed/team_maps.csv'

matches = pd.read_csv(series_result)
matches.drop(columns=['series_link', 'series_result'])
maps = pd.read_csv(maps_stats)



matches["description"] = matches.apply(
    lambda row: f"команда {row['team_name']} сыграла против команды {row['opponent_name']} на карте {row['map_name']}, матч был проведен {row['date']}",
    axis=1,
)



embeddings = model.encode(matches["description"].tolist(), show_progress_bar=True)

collection_match = client.collections.get("MatchData") #fix collections later maybe

def json_clean(obj):
    for k, v in obj.items():
        if isinstance(v, float) and (np.isnan(v) or np.isinf(v)):
            obj[k] = 0.0
    return obj


for i, row in matches.iterrows():
    properties = {
        "team_id": str(row["team_id"]),
        "team_name": row["team_name"],
        "date": row["date"],
        "opponent_id": str(row["opponent_id"]),
        "opponent_name": row["opponent_name"],
        "map_name": row["map_name"],
        "series_result": str(row["series_result"]),
        "description": row["description"],
    }
    properties = json_clean(properties) # had to
    vector = embeddings[i]
    collection_match.data.insert(properties=properties, vector=vector)


Batches: 100%|██████████| 456/456 [00:02<00:00, 169.24it/s]


In [4]:
import re

collection_team_map = client.collections.get("MapStats")
df = pd.read_csv(maps_stats)

def extract_float(value):
    if pd.isna(value):
        return None
    match = re.search(r"[\d.]+", str(value))
    return float(match.group()) if match else None

for col in ['winrate', 'pickrate', 'banrate']:
    df[col] = df[col].apply(extract_float)

for _, row in df.iterrows():
    properties = {
        "team_id": str(row["team_id"]),
        "map_name": row["map_name"],
        "winrate": float(row["winrate"]),
        "pickrate": float(row["pickrate"]),
        "banrate": float(row["banrate"]),
    }
    collection_team_map.data.insert(properties=properties)

In [5]:
from weaviate.classes.query import Filter

match_vectorstore = WeaviateVectorStore(
    client=client,
    index_name="MatchData",  # название класса
    text_key="description",  # по какому полю искать
    embedding=embedding_model,    # эмбеддинги для векторного сравнения
)

match_retriever_certain = match_vectorstore.as_retriever(search_kwargs={"k":1}) # может быть не один, а даже 5, если bo5. думать.
match_retriever_bo3 = match_vectorstore.as_retriever(search_kwargs={"k":3})

In [6]:
router_instructions_ru = """Ты эксперт по классификации запроса пользователя. Твоя задача — направить пользователя к одному из агентов.

Агенты:
- предсказатель результатов матча → "predictor"
- аналитик формы игрока или команды → "shape"
- анализ игрового поведения команды → "team"
- свободные диалоги → "fallback"

В качестве ответа верни **ТОЛЬКО** JSON-объект с ключом "node", одно из значений: "predictor", "shape", "team", "fallback".

**Пример:**
{"node": "predictor"}

Не добавляй никаких объяснений. Не веди диалог. Только JSON.
"""


# возможно стоит добавить bias в сторону fallback, чтобы модель чаще выдавала его для уточнения запроса

In [7]:
local_llm = "ilyagusev/saiga_llama3:latest"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage
import json

test_case_match = llm_json_mode.invoke(
    [SystemMessage(content=router_instructions_ru)]
    + [
        HumanMessage(
            content="кто выиграет в завтрашнем матче: Navi или G2?"
        )
    ]
)
test_case_fallback = llm_json_mode.invoke(
    [SystemMessage(content=router_instructions_ru)]
    + [HumanMessage(content="Я вкусно покушал")]
)
test_case_shape = llm_json_mode.invoke(
    [SystemMessage(content=router_instructions_ru)]
    + [HumanMessage(content="В какой форме находится Niko?")]
)
print(
    json.loads(test_case_match.content),
    json.loads(test_case_fallback.content),
    json.loads(test_case_shape.content),
)

{'node': 'predictor'} {'node': 'fallback'} {'node': 'shape'}


In [9]:
from catboost import CatBoostClassifier

closeness_analyzer_model = CatBoostClassifier()
closeness_analyzer_model.load_model("catboost_series_model.cbm")

sample_feature = pd.DataFrame([[55.0, 10.0, 20.0, 50.0, 8.0, 30.0]], columns=[
    'team_map_winrate', 'team_map_pickrate', 'team_map_banrate',
    'opponent_map_winrate', 'opponent_map_pickrate', 'opponent_map_banrate'
])

# Predict
pred = closeness_analyzer_model.predict(sample_feature)

labels = ["One-sided win", "Close win", "Close loss", "One-sided loss"]
print("Prediction:", labels[int(pred[0])])

Prediction: Close win


/tmp/ipykernel_643280/374847762.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Prediction:", labels[int(pred[0])])


In [10]:
# match predictor section
test_case_prompt = """Твоя задача - извлечь названия команд из запроса пользователя, если они указаны. Если названия команды нет, то укажи 'null'
Ответ должен быть возвращен в формате JSON со следующими ключами:
{
    "team_name": "<название первой команды или null>",
    "opponent_name": "<название второй команды или null>"    
}
Не добавляй никаких комментариев, возвращай чистый JSON-объект.
"""
match_predictor_prompt = """Ты — интеллектуальный парсер пользовательских запросов. Твоя задача — извлечь из текста следующие данные:
В твоей памяти нет абсолютно никаких игр.
🔹 Обязательные данные:
- Название первой команды (team_name)
- Название второй команды (opponent_name)

🔹 Дополнительные данные (если указаны):
- Дата матча (date) — обязательно в формате dd/mm/yyyy (например: 25/11/2024)
- Название карты (map_name)

Ответ должен быть возвращён строго в формате JSON со следующими ключами:

{
  "team_name": "<название первой команды или null>",
  "opponent_name": "<название второй команды или null>",
  "date": "<дата в формате dd/mm/yyyy или null>",
  "map_name": "<название карты или null>"
}

Если какие-либо из данных отсутствуют в пользовательском запросе, укажи значение `null` (это JSON null, без кавычек). Не добавляй никаких комментариев или текста вне JSON — только чистый JSON-объект.
Если названия команды нет, то укажи 'null'
Если названия команды нет, то укажи 'null'
Если названия команды нет, то укажи 'null'
Пример корректного ответа:

{
  "team_name": "Team Spirit",
  "opponent_name": "Natus Vincere",
  "date": "25/11/2024",
  "map_name": "Mirage"
}

Будь внимателен к формулировкам — названия команд, дату и карту могут указывать в свободной форме. Твоя задача — точно извлечь данные и вернуть их в необходимом формате.
"""

test_case_match_pred_1 = llm_json_mode.invoke(
    [SystemMessage(content=match_predictor_prompt)]
    + [
        HumanMessage(
            content="насколько близкой была игра Navi - G2 на карте мираж"
        )
    ]
)
test_case_match_pred_2 = llm_json_mode.invoke(
    [SystemMessage(content=match_predictor_prompt)]
    + [HumanMessage(content="проанализируй игру Team Spirit против Virtus Pro 25 октября 2023 года")]
)
test_case_match_pred_3 = llm_json_mode.invoke(       # fix this ffs
    [SystemMessage(content=match_predictor_prompt)]
    + [HumanMessage(content="Проанализируй вчерашнюю игру")]
)
print(
    json.loads(test_case_match_pred_1.content),
    json.loads(test_case_match_pred_2.content),
    json.loads(test_case_match_pred_3.content),
)

hard_test_case = llm_json_mode.invoke([SystemMessage(content=test_case_prompt)]
    + [HumanMessage(content="Проанализируй вчерашнюю игру")]
)

print(json.loads(hard_test_case.content))

{'team_name': 'Natus Vincere', 'opponent_name': 'G2 Esports', 'date': None, 'map_name': 'Mirage'} {'team_name': 'Team Spirit', 'opponent_name': 'Virtus Pro', 'date': '25/10/2023', 'map_name': None} {'team_name': None, 'opponent_name': None, 'date': 'null', 'map_name': None}
{'team_name': 'null', 'opponent_name': 'null'}


In [11]:
import operator
from typing_extensions import TypedDict
from typing import List, Annotated

class GraphState(TypedDict):
    initial_prompt: str # initial user prompt
    generation: str # LLM generation
    max_retries: int # max number of retries for answering
    answers: int # number of answers generated
    loop_step: Annotated[int, operator.add] # have to use annotated int since using default int will lead into multiple edges not being able to combine values properly (and some other stuff)
    source: List[str] # stats or any other retrieved valuables
    #source: dict
    #extra_source: dict
    extra_source: List[str] #i ve made this source just to be sure it is not overridden, del later or fix

In [30]:
match_doc = match_retriever_certain.invoke('Heroic против G2')
def match_metadata_fetch(match_doc):
    matchdoc_data = match_doc[0].metadata
    team_id = str(re.sub(r'\.\d*', '', matchdoc_data['team_id']))
    opponent_id = re.sub(r'\.\d*', '', matchdoc_data['opponent_id'])
    map_name = matchdoc_data['map_name'] #dont ask why those str turned into float
    return team_id, opponent_id, map_name

team_id, opponent_id, map_name = match_metadata_fetch(match_doc)


def team_map_stats(team_id, opponent_id, map_name):
    response = collection_team_map.query.fetch_objects(
        filters=(Filter.by_property("team_id").equal(team_id)) & Filter.by_property("map_name").equal(map_name)
    )
    if response.objects:
        team1_winrate = response.objects[0].properties['winrate']
        team1_pickrate = response.objects[0].properties['pickrate']
        team1_banrate = response.objects[0].properties['banrate']

    response = collection_team_map.query.fetch_objects(
        filters=(Filter.by_property("team_id").equal(opponent_id)) & Filter.by_property("map_name").equal(map_name)
    )
    if response.objects:
        team2_winrate = response.objects[0].properties['winrate']
        team2_pickrate = response.objects[0].properties['pickrate']
        team2_banrate = response.objects[0].properties['banrate']
    df = pd.DataFrame([[
        team1_winrate,
        team1_pickrate,
        team1_banrate,
        team2_winrate,
        team2_pickrate,
        team2_banrate
    ]], columns=[
        'team_map_winrate',
        'team_map_pickrate',
        'team_map_banrate',
        'opponent_map_winrate',
        'opponent_map_pickrate',
        'opponent_map_banrate'
    ])

    return df

sample_map_stats = team_map_stats(team_id, opponent_id, map_name)

pred = closeness_analyzer_model.predict(sample_map_stats)
labels = ["One-sided win", "Close win", "Close loss", "One-sided loss"]
print("Prediction:", labels[int(pred[0])])

Prediction: Close win


/tmp/ipykernel_643280/85189475.py:50: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Prediction:", labels[int(pred[0])])


In [13]:
# this is query + filter
# it is not working
# why?
# because i fucked up big time
# how?
# by tweaking manual embedding

"""sample_source = {
    "date":'null',
    "map_name":'Mirage'
}

sample_query = 'Heroic против G2 на Mirage'

filter_sample = []
if sample_source['date'] != 'null' or sample_source['date'] != 'None':
    filter_sample.append(Filter.by_property("date").equal(sample_source["date"]))
if sample_source['map_name'] != 'null' or sample_source['map_name'] != 'None':
    filter_sample.append(Filter.by_property("map_name").equal(sample_source["map_name"]))

combined_filter = None
if filter_sample:
    combined_filter = Filter.all_of([*filter_sample])
response = collection_match.query.near_text(
    query=sample_query,
    filters=combined_filter,
    limit=3
)

print(response)"""

'sample_source = {\n    "date":\'null\',\n    "map_name":\'Mirage\'\n}\n\nsample_query = \'Heroic против G2 на Mirage\'\n\nfilter_sample = []\nif sample_source[\'date\'] != \'null\' or sample_source[\'date\'] != \'None\':\n    filter_sample.append(Filter.by_property("date").equal(sample_source["date"]))\nif sample_source[\'map_name\'] != \'null\' or sample_source[\'map_name\'] != \'None\':\n    filter_sample.append(Filter.by_property("map_name").equal(sample_source["map_name"]))\n\ncombined_filter = None\nif filter_sample:\n    combined_filter = Filter.all_of([*filter_sample])\nresponse = collection_match.query.near_text(\n    query=sample_query,\n    filters=combined_filter,\n    limit=3\n)\n\nprint(response)'

In [14]:

# очень глупый и простой промпт 
predictor_final_prompt = """Ты — эксперт по анализу матчей в киберспорте, работающий над задачами вопрос-ответ в рамках интеллектуального агента.

Вот достоверный контекст, который необходимо использовать при формулировании ответа:

{context}

Ниже представлен пользовательский запрос, описывающий интересующий матч:

{user_prompt}

Также предоставлены дополнительные статистические данные по матчу:

{extra_source}

Дополнительные данные содержат числовую статистику по карте в следующем порядке:
- team_map_winrate — винрейт команды на карте
- team_map_pickrate — частота пика карты командой
- team_map_banrate — частота бана карты командой
- opponent_map_winrate — винрейт соперника на карте
- opponent_map_pickrate — частота пика карты соперником
- opponent_map_banrate — частота бана карты соперником

Твоя задача:
1. Принять контекст как полностью достоверную информацию.
2. Использовать статистику из дополнительного источника, чтобы объяснить, почему матч сложился именно так.
3. Учитывать содержание пользовательского запроса при формировании объяснения.
4. Не придумывать информацию и не использовать внешние источники — опирайся только на данные, представленные выше.

Дай чёткий и логичный ответ на русском языке. Максимум 3 предложения. Ответ должен быть кратким и по существу.

Ответ:"""

In [47]:
from langchain.schema import Document #for docs but remove before pushing
from langgraph.graph import END


# nodes
def retrieve_for_match(state):
    # надо найти матч, используя фильтры
    # но фильтры использовать не удалось, потому что я использовал мануал эмбеддинг ранее (хаахахахахахах)
    # и теперь у меня проблема, в одной из закоменченных cells выше о ней написано
    # поэтому мы тупо находим по вектору дескрипшна
    # забивая, к сожалению, на фильтры
    source_raw = match_retriever_certain.invoke(state['initial_prompt'])
    # нашли матч и далее работаем с этим
    matchdoc_data = source_raw[0].metadata # и тут мы берем всего лишь первый объект, хотя мы его берем еще с помощью top k, где k=1 используя retriever_certain
    team_id = str(re.sub(r'\.\d*', '', matchdoc_data['team_id']))
    opponent_id = re.sub(r'\.\d*', '', matchdoc_data['opponent_id'])
    map_name = matchdoc_data['map_name'] #dont ask why those str turned into float
    features_map_stats = team_map_stats(team_id, opponent_id, map_name) # по хорошему нужно рефакторить код, чтобы та функция была где нибудь здесь)
    pred = closeness_analyzer_model.predict(features_map_stats)
    labels = ["Односторонняя победа", "Близкая победа", "Близкое поражение", "Одностороннее поражение"]
    source = labels[int(pred[0])]
    features_map_stats = features_map_stats.iloc[0].to_dict()
    extra_source = features_map_stats
    return {"source": source, "extra_source": extra_source}

def predict_match_first(state):
    # пользователь задал вопрос и попал в эту ветку
    # для начала необходимо проверить полноту данных для проведения аналитики, в нашем случае нам нужны: 
    #   названия двух команд, 
    #   дата
    #       если не указана, то берется любая (исправить на самую недавнюю в дальнейшем)
    #       (в дальнейшем добавить) если дата указана, но матча нет, то написать об этом пользователю (извлечение и преобразование даты с помощью ллм и дальнейшее использование фильтра weaviate)
    #   карта
    #       если карта не указана, то берется топ3 к и перечисляется пользователю для уточнения (todo++++)
    #       если карта указана и указана дата, но матча не нашлось, то написать о том, что данный матч не найден
    # Для начала ллм определит указана ли карта и дата в запросе пользователя
    # далее проверяем наличие данных по запросу и отдаем ответ
    # 
    #initial_prompt = state["initial_prompt"]
    match_predictor_data_check = llm_json_mode.invoke(
    [SystemMessage(content=match_predictor_prompt)]
    + [HumanMessage(content=state['initial_prompt'])])
    source = json.loads(match_predictor_data_check.content)
    return {"source": source}
        #return "proceed_match_predict"
    #source = match_metadata_fetch(source)
    # сначала здесь проверить полноту данных для проведения аналитики (данные, которые предоставил пользователь)
    # далее вызвать функцию инференса модели, написанной для аналитики
    # далее передать вопрос пользователя и ответ модели в виде initial prompt = question и source = context
    print("произошел предикт матча")
    #initial_prompt = state['initial_prompt']
    #source = state["source"]
    #loop_step = state.get("loop_step, 0")
    #return {"generation: generation, "loop_step": loop_step + 1}
    # pretty much the same goes for other analyzers, the only thing is the way it generates using premade prompt

def predict_match_second(state):
    # на этом этапе мы знаем что пользователь дал более менее корректный запрос
    initial_prompt = state['initial_prompt']
    source = state['source']
    extra_source = state['extra_source']
    formatted_prompt = predictor_final_prompt.format(
        context=source,
        user_prompt=initial_prompt,
        extra_source=json.dumps(extra_source, ensure_ascii=False))
    generation = llm.invoke([HumanMessage(content=formatted_prompt)])
    return {"generation": generation.content}

def rejector(state):
    print('Не были указаны названия команд (или было указано название только для одной команды)\nПовторите ваш запрос, указав названия команд')

def analyze_shape(state):
    print("произошла аналитика формы игрока или команды")

def analyze_behaviour(state):
    print("произошла аналитика внутриигрового поведения команды")

def fallback(state):
    # should finish the generation with proper apology
    print('извинитесь за ваш запрос')

def decide_to_generate_predict_match(state):
    """
    Функция решает достаточно ли данных пользователь предоставил для проведения аналитики
    Если данных недостаточно, то ссылается на другую функцию, которая ищет данные в датасторе или бд
    (возможно стоит доработать, чтобы данные проверялись на правдивость)
    Функция возвращает булево значение.
    """
    initial_prompt = state["initial_prompt"]
    filtered_documents = state["documents"]

    




# edges
def route_questions(state):
    route_question = llm_json_mode.invoke(
        [SystemMessage(content=router_instructions_ru)]
        + [HumanMessage(content=state["initial_prompt"])]
    )
    source = json.loads(route_question.content)["node"]
    if source == "predictor":
        return "predict_match"
    elif source == "shape":
        return "analyze_shape"
    elif source == "team":
        return "analyze_behaviour"
    elif source == "fallback":
        return "fallback"


def decide_match_predict_route(state):
    source = state["source"]

    if source['team_name'] in ['null', 'None'] or source['opponent_name'] in ['null', 'None']:
        return "rejector"
    else:
        return "proceed_match_predict"



hallucination_grader_instructions = ''

# do not connect until graders are finished
def grade_generation(state):
    """ Decide whether the output is hallucinated or based on stats. Then determine if it is useful or not.
    """
    initial_prompt = state['initial_prompt']
    source = state['source']
    generation = state['generation']
    max_retries = state.get('max_retries', 2) # default to 2 retries

    hallucination_grader_instructions_formatted = hallucination_grader_instructions.format(
        source = 'smth', generation=generation.content
    )
    result = llm_json_mode.invoke(
        [SystemMessage(content=hallucination_grader_instructions)]
        + [HumanMessage(content=hallucination_grader_instructions_formatted)]
    )
    grade = json.loads(result.content)["binary_score"]
    

    if grade == 'yes':
        # check here if generation is full enough, useful and answers the question properly
        # similar to hallucination grader but with different prompt and output will be binary yes or no again
        if grade == 'yes':
            return 'useful'
        elif state['loop_step'] <= max_retries:
            return 'not_useful'
        else:
            return "max retries" #means model couldnt answer the question properly in given max retries
    elif state['loop_step'] <= max_retries:
        return 'hallucinated' # hallucinated therefore couldnt get useful stats and generated on their own or smth so this will be regeneration attempt
    else:
        return "max retries"


In [48]:
from langgraph.graph import StateGraph
from IPython.display import Image, display

workflow = StateGraph(GraphState)

workflow.add_node("retrieve_for_match", retrieve_for_match)  # retriever
workflow.add_node("predict_match", predict_match_first)  # predictor
workflow.add_node("analyze_shape", analyze_shape)
workflow.add_node("analyze_behaviour", analyze_behaviour)
workflow.add_node("fallback", fallback)
workflow.add_node('rejector', rejector)
workflow.add_node('predict_match_second', predict_match_second)

workflow.set_conditional_entry_point(
    route_questions,
    {
        "predict_match": "predict_match",
        "analyze_shape": "analyze_shape",
        "analyze_behaviour" : "analyze_behaviour",
        "fallback" : "fallback"
    },
)


#

workflow.add_conditional_edges(
    "predict_match",
    decide_match_predict_route,
    {
        "rejector": "rejector",
        "proceed_match_predict": "retrieve_for_match",
    },
)

workflow.add_edge("predict_match", "retrieve_for_match")
workflow.add_edge('retrieve_for_match', 'predict_match_second')

# Compile
graph = workflow.compile()
#display(Image(graph.get_graph().draw_mermaid_png())) #not displaying cuz not loading mermaid LOL


In [49]:
inputs = {"initial_prompt": "ПРивет я коза", "max_retries": 3}
for event in graph.stream(inputs, stream_mode="values"):
    print(event)
#inputs = {"initial_prompt": "Расскажи о завтрашнем матче Navi vs Falcons", "max_retries": 3}
#for event in graph.stream(inputs, stream_mode="values"):
#    print(event)

inputs = {"initial_prompt": "Расскажи о матче heroic против Cloud9 на карте Nuke 24/11/23", "max_retries": 3}
for event in graph.stream(inputs, stream_mode="values"):
    print(event) 
inputs = {"initial_prompt": "Хорошо ли сейчас играет Niko", "max_retries": 3}
for event in graph.stream(inputs, stream_mode="values"):
    print(event)

{'initial_prompt': 'ПРивет я коза', 'max_retries': 3, 'loop_step': 0}
извинитесь за ваш запрос
{'initial_prompt': 'Расскажи о матче heroic против Cloud9 на карте Nuke 24/11/23', 'max_retries': 3, 'loop_step': 0}
{'initial_prompt': 'Расскажи о матче heroic против Cloud9 на карте Nuke 24/11/23', 'max_retries': 3, 'loop_step': 0, 'source': {'team_name': 'Heroic', 'opponent_name': 'Cloud9', 'date': '24/11/2023', 'map_name': 'Nuke'}}
{'initial_prompt': 'Расскажи о матче heroic против Cloud9 на карте Nuke 24/11/23', 'max_retries': 3, 'loop_step': 0, 'source': 'Односторонняя победа', 'extra_source': {'team_map_winrate': 41.9, 'team_map_pickrate': 14.7, 'team_map_banrate': 62.1, 'opponent_map_winrate': 64.3, 'opponent_map_pickrate': 43.3, 'opponent_map_banrate': 8.6}}


/tmp/ipykernel_643280/14933167.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  source = labels[int(pred[0])]


{'initial_prompt': 'Расскажи о матче heroic против Cloud9 на карте Nuke 24/11/23', 'generation': 'Матч между командами heroic и Cloud9 на карте Nuke 24 ноября 2023 года сложился так, потому что Cloud9 имели более высокий винрейт (64.3%) и частоту пика карты (43.3%), что дало им преимущество в игре. Это, вероятно, способствовало односторонней победе Cloud9.', 'max_retries': 3, 'loop_step': 0, 'source': 'Односторонняя победа', 'extra_source': {'team_map_winrate': 41.9, 'team_map_pickrate': 14.7, 'team_map_banrate': 62.1, 'opponent_map_winrate': 64.3, 'opponent_map_pickrate': 43.3, 'opponent_map_banrate': 8.6}}
{'initial_prompt': 'Хорошо ли сейчас играет Niko', 'max_retries': 3, 'loop_step': 0}
произошла аналитика формы игрока или команды
